# Train fcn8-atonce from vgg16

In [1]:
import caffe
# source: http://nbviewer.jupyter.org/github/BVLC/caffe/blob/master/examples/net_surgery.ipynb

# Load the original network and extract the fully connected layers' parameters.

In [2]:
net = caffe.Net('vgg16-ILSVRC-2012/deploy.prototxt', 
                'vgg16-ILSVRC-2012/VGG_ILSVRC_16_layers.caffemodel', 
                caffe.TEST)
params = ['fc6', 'fc7', 'fc8']
# fc_params = {name: (weights, biases)}
fc_params = {pr: (net.params[pr][0].data, net.params[pr][1].data) for pr in params}

for fc in params:
    print('{} weights are {} dimensional and biases are {} dimensional'.format(fc, fc_params[fc][0].shape, fc_params[fc][1].shape))

fc6 weights are (4096, 25088) dimensional and biases are (4096,) dimensional
fc7 weights are (4096, 4096) dimensional and biases are (4096,) dimensional
fc8 weights are (1000, 4096) dimensional and biases are (1000,) dimensional


# Load the fully convolutional network to transplant the parameters.

In [3]:
net_full_conv = caffe.Net('city-fcn8s-atonce/deploy.prototxt', 
                          'vgg16-ILSVRC-2012/VGG_ILSVRC_16_layers.caffemodel',
                          caffe.TEST)
params_full_conv = ['fc6-conv', 'fc7-conv']
# conv_params = {name: (weights, biases)}
conv_params = {pr: (net_full_conv.params[pr][0].data, net_full_conv.params[pr][1].data) for pr in params_full_conv}

for conv in params_full_conv:
    print('{} weights are {} dimensional and biases are {} dimensional'.format(conv, conv_params[conv][0].shape, conv_params[conv][1].shape))

fc6-conv weights are (4096, 512, 7, 7) dimensional and biases are (4096,) dimensional
fc7-conv weights are (4096, 4096, 1, 1) dimensional and biases are (4096,) dimensional


# Transplant & save

In [4]:
for pr, pr_conv in zip(params, params_full_conv):
    conv_params[pr_conv][0].flat = fc_params[pr][0].flat  # flat unrolls the arrays
    conv_params[pr_conv][1][...] = fc_params[pr][1]
    
net_full_conv.save('city-fcn8s-atonce/vgg16fc-CITYSCAPES.caffemodel')